In [77]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../classes')
sys.path.append('../modules')
from BuildSpecimen import build_specimens_output
from PlotFitTesting import plot_fit_test_data, plot_test_specimens
from PerformTTest import perform_t_test
import os
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Dedicated to Using ML on Specimen Data

In [67]:
#Reading in Coupond Data Sheet
cwd = os.getcwd()
pardir = os.path.dirname(os.path.dirname(cwd))
specimens_fp = os.path.join(pardir,'CouponData','CouponIDDataSheetReformat.xlsx')
specimens_df = build_specimens_output(specimens_fp)

In [103]:
#Building Random Forest for Max Load Given Description and Postlaminated Weight

#Defining X and y
exclude_descriptions = ['2 Piece balsa stringer Blue on Top',
                        'Solid Balsa Stringer- Slightly Damaged by plainar in dib Blue on Top',
                        'Solid Balsa Stringer Blue on Top',
                        'China Balsa Blue on Top Foam Shear Fail']

features_target_df = specimens_df[~specimens_df['Description'].isin(exclude_descriptions)][['PreLaminated Weight (g)','PostLaminated Weight (g)','Description','Max Load (lb)']].dropna()

X = features_target_df.drop(columns='Max Load (lb)')
y = features_target_df['Max Load (lb)']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)

# Function to compute the difference between columns and reshape the output
def compute_weight_difference(X):
    weight_difference = X[:, 1] - X[:, 0]
    return np.reshape(weight_difference, (-1, 1))

# Create a ColumnTransformer to handle encoding and transformation
preprocessor = ColumnTransformer(
    transformers=[
        ('encoder', OneHotEncoder(handle_unknown='ignore'), ['Description']),
        ('scaler', StandardScaler(), ['PostLaminated Weight (g)'])
    ])

# Create the pipeline with preprocessor and random forest regressor
RFpipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=2))
])

RFpipeline.fit(X_train, y_train)

# Predict using the test data
y_pred = RFpipeline.predict(X_test)

# Calculate the root mean squared error
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Root Mean Squared Error pounds:", rmse)

Root Mean Squared Error pounds: 32.81244586561419
